<a href="https://colab.research.google.com/github/khare19yash/Automated-Detection-of-Neuropsychiatric-Disorders/blob/master/ASD_DiagNet_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [2]:
cd gdrive/My Drive/IIITH_Internship/Neuro/ASD

/content/gdrive/My Drive/IIITH_Internship/Neuro/ASD


### Downloading AAL data

In [ ]:
!pip install nilearn
import nilearn
nilearn.datasets.fetch_abide_pcp(data_dir='./Dataset', n_subjects=None, pipeline='cpac', band_pass_filtering=True, global_signal_regression=True, derivatives = 'rois_aal', quality_checked=True)

### Declaring variables

In [3]:
#options: cc200, dosenbach160, aal
p_ROI = "cc200-ho"
p_fold = 10
p_center = "Stanford"
p_mode = "whole"
p_augmentation = False
p_Method = "ASD-DiagNet"

In [4]:
parameter_list = [p_ROI,p_fold,p_center,p_mode,p_augmentation,p_Method]
print("*****List of patameters****")
print("ROI atlas: ",p_ROI)
print("per Center or whole: ",p_mode)
if p_mode == 'percenter':
    print("Center's name: ",p_center)
print("Method's name: ",p_Method)
if p_Method == "ASD-DiagNet":
    print("Augmentation: ",p_augmentation)

*****List of patameters****
ROI atlas:  cc200-ho
per Center or whole:  whole
Method's name:  ASD-DiagNet
Augmentation:  False


### Installing Wandb and pyprind

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.7MB 2.8MB/s 
     |████████████████████████████████| 122kB 18.0MB/s 
     |████████████████████████████████| 163kB 16.9MB/s 
     |████████████████████████████████| 102kB 8.5MB/s 
     |████████████████████████████████| 102kB 7.5MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=b4de7bf67230ac037fc303415436d6c96b624b58029648094075d4d20d0ecf31
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=e05d17e76f49765317e75cbbf9252e066de79f96ac6b5889e4cceb6a0b1811bf
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=7c3a11ad68c598836473fd8b87518e73d930d2b8ce2245

In [ ]:
!pip install pyprind

### Importing Libraries

In [5]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import reduce
from sklearn.impute import SimpleImputer
import time
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import pyprind
import sys
import pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold, StratifiedKFold
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from scipy import stats
from sklearn import tree
import functools
import numpy.ma as ma # for masked arrays
import pyprind
import random
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import wandb
from sklearn.decomposition import PCA


# !wandb login 93ff2b234de4902b71d7405831d980f0c3182650

ModuleNotFoundError: ignored

In [ ]:
!wandb login 93ff2b234de4902b71d7405831d980f0c3182650

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Importing the data 

In [ ]:
def get_key(filename):
    f_split = filename.split('_')
    if f_split[3] == 'rois':
        key = '_'.join(f_split[0:3]) 
    else:
        key = '_'.join(f_split[0:2])
    return key

In [6]:
cc200 = pd.read_csv('./asd-cc200-all.csv')
# flist = cc200[cc200['n_timepoints'] >= 1]['FILE_ID']        # Only for DFC
flist = cc200['FILE_ID']
len(flist)

1035

In [7]:
data_main_cc200 = './Datasets/ASD_Dataset/CC200'   
data_main_cc400 = './Datasets/ASD_Dataset/CC400' 
data_main_ho = './Datasets/ASD_Dataset/HO'    #cc200'#path to time series data
# flist = os.listdir(data_main_path)
cc200 = pd.read_csv('./asd-cc200-all.csv')
# flist = cc200[cc200['n_timepoints'] >= 116]['FILE_ID']
flist = cc200['FILE_ID']
flist = flist.tolist()
print(len(flist))

labels = {}
for row in cc200.iterrows():
    file_id = row[1]['FILE_ID']
    y_label = row[1]['DX_GROUP']
    assert(file_id not in labels)
    if(file_id in flist) :
        labels[file_id] = y_label
    
print(len(labels))  

1035
1035


In [ ]:
len(os.listdir(data_main_ho))

1035

### Only in case of DFC

In [ ]:
def get_corr(df):
    
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        return corr
    
def get_dfcWVar(filename, window = 30, stride = 10, phase = 'matrix'):
    
    """
    Returns the upper triangular part of Variance of dfc matrix extracted using weighted window approach.
    phase : matrix (if u need a correlation matrix)
          : 'triu' (if u need upper triangular matrix of correlation matrix)
    inputs : timeseries array     (T X n_rois)
    output : correleation matrix  (n_windows X n_rois X n_rois)
    """
    for file in os.listdir(data_main_path):
        if file.startswith(filename):
            data = pd.read_csv(os.path.join(data_main_path, file), sep='\t')
            break  
    data = np.array(data)
    
    n_samples, n_rois = data.shape
    dfc = []
    size = n_samples - window + 1
    
    for i in range(0, size, stride) :
        # print(i)
        df = data[i : i + window , : ]
        weights = np.linspace(1 / n_samples, window / n_samples, num = window) # Linearly distributed weights(Weighted Window)
        weights = weights[:,np.newaxis]       # run this only in case of weighted window(not for tapered window)
        df = weights * df
        corr = get_corr(df)
        dfc.append(corr)
    dfc = np.stack(dfc, axis = 0)
    dfcvar = np.var(dfc, axis = 0) 
    if(phase == 'triu') :
            mask = np.invert(np.tri(dfcvar.shape[0], k=-1, dtype=bool))
            m = ma.masked_where(mask == 1, mask)
            dfcvar = ma.masked_where(m, dfcvar).compressed()
            
    return dfcvar

### Helper functions for computing correlations

In [ ]:
# def get_corr_data(filename):
#     # returns upper triangular correlation vector
#     for file in os.listdir(data_main_cc200):
#         if file.startswith(filename):
#             df200 = pd.read_csv(os.path.join(data_main_cc200, file), sep='\t')
#     with np.errstate(invalid="ignore"):
#         corr = np.nan_to_num(np.corrcoef(df200.T))
#         mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
#         m = ma.masked_where(mask == 1, mask)
#         fc200 = ma.masked_where(m, corr).compressed()

#     # for file in os.listdir(data_main_cc400):
#     #     if file.startswith(filename):
#     #         df400 = pd.read_csv(os.path.join(data_main_cc400, file), sep='\t')
            
#     # with np.errstate(invalid="ignore"):
#     #     corr = np.nan_to_num(np.corrcoef(df400.T))
#     #     mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
#     #     m = ma.masked_where(mask == 1, mask)
#     #     fc400 =  ma.masked_where(m, corr).compressed()

#     for file in os.listdir(data_main_ho):
#         if file.startswith(filename):
#             dfho = pd.read_csv(os.path.join(data_main_ho, file), sep='\t')
            
#     with np.errstate(invalid="ignore"):
#         corr = np.nan_to_num(np.corrcoef(dfho.T))
#         mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
#         m = ma.masked_where(mask == 1, mask)
#         fcho =  ma.masked_where(m, corr).compressed()

#     return np.concatenate([fcho, fc200], axis = 0)
#     # return fc400

def get_corr_data(filename, data_paths):
    # returns upper triangular correlation vector
    fcs = []
    for path in data_paths : 
        for file in os.listdir(path):
            if file.startswith(filename):
                df = pd.read_csv(os.path.join(path, file), sep='\t')
        with np.errstate(invalid="ignore"):
            corr = np.nan_to_num(np.corrcoef(df.T))
            mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
            m = ma.masked_where(mask == 1, mask)
            fc = ma.masked_where(m, corr).compressed()
        fcs.append(fc)
    fcs = np.concatenate(fcs, axis = 0)

    return fcs

def get_corr_matrix(filename):
    # returns correlation matrix
    for file in os.listdir(data_main_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_main_path, file), sep='\t')
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        return corr


def get_label(filename):
    assert (filename in labels)
    return labels[filename]

def confusion(g_turth,predictions):
    tn, fp, fn, tp = confusion_matrix(g_turth,predictions).ravel()
    accuracy = (tp+tn)/(tp+fp+tn+fn)
    sensitivity = (tp)/(tp+fn)
    specificty = (tn)/(tn+fp)
    return accuracy,sensitivity,specificty

def get_regs(samplesnames,regnum):
    # returns region index array
    datas = []
    for sn in samplesnames:
        datas.append(all_corr[sn][0])
    datas = np.array(datas)     # datas shape len(samplesnames)x19900
    avg=[]
    for ie in range(datas.shape[1]):
        avg.append(np.mean(datas[:,ie]))
    avg=np.array(avg)
    highs=avg.argsort()[-regnum:][::-1]
    lows=avg.argsort()[:regnum][::-1]
    regions=np.concatenate((highs,lows),axis=0) # shape (9950,)
    return regions
  
def get_corr_arr(flist, paths, p_ROI):
    corr_arr = []
    all_corr = get_corr(paths, p_ROI)
    for f in flist:
        corr_arr.append(all_corr[f][0])

    corr_arr = np.stack(corr_arr,axis=0)    
    return corr_arr


In [ ]:
filename =  flist[0]
df = get_corr_data(filename, [data_main_ho, data_main_cc200])
df.shape

(26005,)

### Computing SFC

In [ ]:
# all_corr dictionary with key - filename and value - (correlation_data(vector of size 19,900),label)
def get_corr(paths, p_ROI):
    if not os.path.exists('./correlations_file_triu'+p_ROI+'.pkl'):
        pbar=pyprind.ProgBar(len(flist))
        all_corr = {}
        for f in flist:
          
            lab = get_label(f)
            all_corr[f] = (get_corr_data(f, paths), lab)
            pbar.update()

        print('Corr-computations finished')

        pickle.dump(all_corr, open('./correlations_file_triu'+p_ROI+'.pkl', 'wb'))
        print('Saving to file finished')

    else:
        all_corr = pickle.load(open('./correlations_file_triu'+p_ROI+'.pkl', 'rb'))

    return all_corr

### Using PCA on combined atlas

In [ ]:
corr_arr_cc200 = get_corr_arr(flist, [data_main_cc200], 'cc200')
corr_arr_ho = get_corr_arr(flist, [data_main_ho], 'ho')
print(corr_arr_cc200.shape)
print(corr_arr_ho.shape)

(1035, 19900)
(1035, 6105)


In [ ]:
pca = PCA(n_components=1024)
corr_arr_cc200 = pca.fit_transform(corr_arr_cc200)
corr_arr_ho = pca.fit_transform(corr_arr_ho)

In [ ]:
print(corr_arr_cc200.shape)
print(corr_arr_ho.shape)

(1035, 1024)
(1035, 1024)


In [ ]:
corr_arr = np.concatenate([corr_arr_cc200, corr_arr_ho], axis = 1)
corr_arr.shape

(1035, 2048)

In [ ]:
# train_correlations_file dictionary with key - filename and value - (correlation_data(vector of size 19,900),label)
if not os.path.exists('/correlations_file_separate_pca1024_'+p_ROI+'.pkl'):
    pbar=pyprind.ProgBar(len(flist))
    all_corr = {}


    for i,f in enumerate(flist):
        all_corr[f] = (corr_arr[i], get_label(f))
        pbar.update()

    print('Corr-computations finished')

    pickle.dump(all_corr, open('/correlations_file_separate_pca1024_'+p_ROI+'.pkl', 'wb'))
    print('Saving to file finished')

else:
    all_corr = pickle.load(open('/correlations_file_separate_pca1024_'+p_ROI+'.pkl', 'rb'))

0% [##############################] 100% | ETA: 00:00:00

Corr-computations finished
Saving to file finished



Total time elapsed: 00:00:00


In [ ]:
print(all_corr[flist[0]][0].shape)
print(len(all_corr))

(2048,)
1035


### To extract sfc instead of upper triangular matrices

In [ ]:
all_corr = {}
for f in flist:

    lab = get_label(f)
    all_corr[f] = (get_corr_matrix(f), lab)

print('Corr-computations finished')

Corr-computations finished


## Computing eigenvalues and eigenvector

In [ ]:
if p_Method=="ASD-DiagNet":
    eig_data = {}
    pbar = pyprind.ProgBar(len(flist))
    for f in flist:  
        d = get_corr_matrix(f)
        eig_vals, eig_vecs = np.linalg.eig(d)

        for ev in eig_vecs.T:
            np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))

        sum_eigvals = np.sum(np.abs(eig_vals))
        # Make a list of (eigenvalue, eigenvector, norm_eigval) tuples
        eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i], np.abs(eig_vals[i])/sum_eigvals)
                     for i in range(len(eig_vals))]

        # Sort the (eigenvalue, eigenvector) tuples from high to low
        eig_pairs.sort(key=lambda x: x[0], reverse=True)

        eig_data[f] = {'eigvals':np.array([ep[0] for ep in eig_pairs]),
                       'norm-eigvals':np.array([ep[2] for ep in eig_pairs]),
                       'eigvecs':[ep[1] for ep in eig_pairs]}
        pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:31


## Calculating Eros similarity

In [ ]:
def norm_weights(sub_flist):
    num_dim = len(eig_data[flist[0]]['eigvals'])
    norm_weights = np.zeros(shape=num_dim)
    for f in sub_flist:
        norm_weights += eig_data[f]['norm-eigvals'] 
    return norm_weights

def cal_similarity(d1, d2, weights, lim=None):
    res = 0.0
    if lim is None:
        weights_arr = weights.copy()
    else:
        weights_arr = weights[:lim].copy()
        weights_arr /= np.sum(weights_arr)
    for i,w in enumerate(weights_arr):
        res += w*np.inner(d1[i], d2[i])
    return res

In [ ]:
all_corr[flist[0]][0].shape

(19900,)

## Defining dataset class

In [ ]:
class CC200Dataset(Dataset):
    def __init__(self, pkl_filename=None, data=None, samples_list=None, 
                 augmentation=False, aug_factor=1, num_neighbs=5,
                 eig_data=None, similarity_fn=None, verbose=False,regs=None):
        
        self.regs=regs
        if pkl_filename is not None:
            if verbose:
                print ('Loading ..!', end=' ')
            self.data = pickle.load(open(pkl_filename, 'rb'))
        elif data is not None:
            self.data = data.copy()
            
        else:
            sys.stderr.write('Either PKL file or data is needed!')
            return 

        #if verbose:
        #    print ('Preprocess..!', end='  ')
        if samples_list is None:
            self.flist = [f for f in self.data]
        else:
            self.flist = [f for f in samples_list]
        self.labels = np.array([self.data[f][1] for f in self.flist])
        
        current_flist = np.array(self.flist.copy())
        current_lab0_flist = current_flist[self.labels == 0]
        current_lab1_flist = current_flist[self.labels == 1]
        #if verbose:
        #    print(' Num Positive : ', len(current_lab1_flist), end=' ')
        #    print(' Num Negative : ', len(current_lab0_flist), end=' ')
        
        
        if augmentation:
            self.num_data = aug_factor * len(self.flist)
            self.neighbors = {}
            pbar = pyprind.ProgBar(len(self.flist))
            weights = norm_weights(samples_list)#??
            for f in self.flist:
                label = self.data[f][1]
                candidates = (set(current_lab0_flist) if label == 0 else set(current_lab1_flist))
                candidates.remove(f)
                eig_f = eig_data[f]['eigvecs']
                sim_list = []
                for cand in candidates:
                    eig_cand = eig_data[cand]['eigvecs']
                    sim = similarity_fn(eig_f, eig_cand,weights)
                    sim_list.append((sim, cand))
                sim_list.sort(key=lambda x: x[0], reverse=True)
                self.neighbors[f] = [item[1] for item in sim_list[:num_neighbs]]#list(candidates)#[item[1] for item in sim_list[:num_neighbs]]
        
        else:
            self.num_data = len(self.flist)

        
    def __getitem__(self, index):
        if index < len(self.flist):
            fname = self.flist[index]
            data = self.data[fname][0].copy() #get_corr_data(fname, mode=cal_mode)    
            # data = data[self.regs].copy()
            label = (self.labels[index],)
            return torch.FloatTensor(data), torch.FloatTensor(label)
        else:
            f1 = self.flist[index % len(self.flist)]
            d1, y1 = self.data[f1][0], self.data[f1][1]
            # d1=d1[self.regs]
            f2 = np.random.choice(self.neighbors[f1])
            d2, y2 = self.data[f2][0], self.data[f2][1]
            # d2=d2[self.regs]
            assert y1 == y2
            r = np.random.uniform(low=0, high=1)
            label = (y1,)
            data = r*d1 + (1-r)*d2
            return torch.FloatTensor(data), torch.FloatTensor(label)

    def __len__(self):
        return self.num_data

### Data Augmentation by flipping the signs of the input

In [ ]:
class CC200Dataset(Dataset):
    def __init__(self, pkl_filename=None, data=None, samples_list=None, 
                 augmentation=False, aug_factor=1, num_neighbs=5,
                 eig_data=None, similarity_fn=None, verbose=False,regs=None):
        
        self.regs=regs
        if pkl_filename is not None:
            if verbose:
                print ('Loading ..!', end=' ')
            self.data = pickle.load(open(pkl_filename, 'rb'))
        elif data is not None:
            self.data = data.copy()
            
        else:
            sys.stderr.write('Either PKL file or data is needed!')
            return 

        #if verbose:
        #    print ('Preprocess..!', end='  ')
        if samples_list is None:
            self.flist = [f for f in self.data]
        else:
            self.flist = [f for f in samples_list]
        self.labels = np.array([self.data[f][1] for f in self.flist])
        
        current_flist = np.array(self.flist.copy())
        current_lab0_flist = current_flist[self.labels == 0]
        current_lab1_flist = current_flist[self.labels == 1]
        #if verbose:
        #    print(' Num Positive : ', len(current_lab1_flist), end=' ')
        #    print(' Num Negative : ', len(current_lab0_flist), end=' ')
        
        
        if augmentation:
            self.num_data = aug_factor * len(self.flist)
            # self.neighbors = {}
            # pbar = pyprind.ProgBar(len(self.flist))
            # weights = norm_weights(samples_list)#??
            # for f in self.flist:
            #     label = self.data[f][1]
            #     candidates = (set(current_lab0_flist) if label == 0 else set(current_lab1_flist))
            #     candidates.remove(f)
            #     eig_f = eig_data[f]['eigvecs']
            #     sim_list = []
            #     for cand in candidates:
            #         eig_cand = eig_data[cand]['eigvecs']
            #         sim = similarity_fn(eig_f, eig_cand,weights)
            #         sim_list.append((sim, cand))
            #     sim_list.sort(key=lambda x: x[0], reverse=True)
            #     self.neighbors[f] = [item[1] for item in sim_list[:num_neighbs]]#list(candidates)#[item[1] for item in sim_list[:num_neighbs]]
        
        else:
            self.num_data = len(self.flist)

        
    def __getitem__(self, index):
        if index < len(self.flist):
            fname = self.flist[index]
            data = self.data[fname][0].copy() #get_corr_data(fname, mode=cal_mode)    
            # data = data[self.regs].copy()
            label = (self.labels[index],)
            return torch.FloatTensor(data), torch.FloatTensor(label)
        else:
            f1 = self.flist[index % len(self.flist)]
            d1, y1 = self.data[f1][0], self.data[f1][1]
            # d1=d1[self.regs]
            label = (y1,)
            data = (-1) * d1
            return torch.FloatTensor(data), torch.FloatTensor(label)

    def __len__(self):
        return self.num_data

## Definig data loader function

In [ ]:
def get_loader(pkl_filename=None, data=None, samples_list=None,
               batch_size=64, 
               num_workers=1, mode='train',
               *, augmentation=False, aug_factor=1, num_neighbs=5,
                 eig_data=None, similarity_fn=None, verbose=False,regions=None):
    """Build and return data loader."""
    if mode == 'train':
        shuffle = True
    else:
        shuffle = False
        augmentation=False

    dataset = CC200Dataset(pkl_filename=pkl_filename, data=data, samples_list=samples_list,
                           augmentation=augmentation, aug_factor=aug_factor, 
                           eig_data=eig_data, similarity_fn=similarity_fn, verbose=verbose,regs=regions)

    data_loader = DataLoader(dataset,
                             batch_size=batch_size,
                             shuffle=shuffle,
                             num_workers=num_workers)
  
    return data_loader

## Defining Autoencoder class

In [ ]:
class MTAutoEncoder(nn.Module):
    def __init__(self, num_inputs=990, 
                 num_latent=200, tied=True,
                 num_classes=2, use_dropout=False):
        super(MTAutoEncoder, self).__init__()
        self.tied = tied
        self.num_latent = num_latent
        
        self.fc_encoder = nn.Linear(num_inputs, num_latent)
    
        if not tied:
            self.fc_decoder = nn.Linear(num_latent, num_inputs)
         
        self.fc_encoder = nn.Linear(num_inputs, num_latent)
        
        if use_dropout:
            self.classifier = nn.Sequential (
                nn.Dropout(p=0.5),
                nn.Linear(self.num_latent, 1),
                
            )
        else:
            self.classifier = nn.Sequential (
                nn.Linear(self.num_latent, 1),
            )
            
         
    def forward(self, x, eval_classifier=False):
        x = self.fc_encoder(x)
        x = torch.tanh(x)
        if eval_classifier:
            x_logit = self.classifier(x)
        else:
            x_logit = None
        
        if self.tied:
            x = F.linear(x, self.fc_encoder.weight.t())
        else:
            x = self.fc_decoder(x)
            
        return x, x_logit

mtae = MTAutoEncoder()

mtae

MTAutoEncoder(
  (fc_encoder): Linear(in_features=990, out_features=200, bias=True)
  (classifier): Sequential(
    (0): Linear(in_features=200, out_features=1, bias=True)
  )
)

### First ASD CNN Model

In [ ]:
class Model(nn.Module):
    def __init__(self,conv_kernel_size,pool_kernel_size,drop_prob,num_classes=2):
        super(Model,self).__init__()

        self.conv_kernel_size = conv_kernel_size
        self.pool_kernel_size = pool_kernel_size
        self.num_classes = num_classes
        self.drop_prob = drop_prob

        self.conv2d = []
        self.max_pool = []

        for conv_kernel, pool_kernel in zip(self.conv_kernel_size, self.pool_kernel_size):
          self.conv2d.append(nn.Conv2d(6, 400, conv_kernel, stride=1))
          self.max_pool.append(nn.MaxPool2d(pool_kernel, stride=(1,1)))

        self.conv2d = nn.ModuleList(self.conv2d)
        self.max_pool = nn.ModuleList(self.max_pool)

        self.dropout = nn.Dropout2d(self.drop_prob)
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(2800,self.num_classes)

    def forward(self, x):
        # forward pass
        conv_pool_out = []
        for conv, pool in zip(self.conv2d, self.max_pool):
            output = pool(torch.tanh(conv(x)))
            conv_pool_out.append(output.squeeze(3))
        out = torch.cat(conv_pool_out,2)
        if self.drop_prob is not None:
            out = self.dropout(out)
        out = self.flatten(out)
        out = self.linear(out)

        return out
# print(model)
# model.to(device)

# wandb.watch(model, log='all')      # this logs everything in wandb account

# optimizer = optim.Adam(model.parameters(),lr=config.lr)

## Defining training and testing functions

In [ ]:
def train(model, epoch, train_loader, p_bernoulli=None, mode='both', lam_factor=1.0):
    model.train()
    train_losses = []
    clf_train_loss = []
    ae_train_loss = []
#     bar = tqdm(train_loader, leave = False)
    
    if mode in ['both', 'clf']:
        final_targets = []
        final_predictions = []
    else:
        final_targets = None
        final_predictions = None    
    
    for i,(batch_x,batch_y) in enumerate(train_loader):
        if len(batch_x) != batch_size:
            continue
        if p_bernoulli is not None:
            if i == 0:
                p_tensor = torch.ones_like(batch_x).to(device)*p_bernoulli
            rand_bernoulli = torch.bernoulli(p_tensor).to(device)

        data, target = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()

        if mode in ['both', 'ae']:
            if p_bernoulli is not None:
                rec_noisy, _ = model(data*rand_bernoulli, False)
                loss_ae = criterion_ae(rec_noisy, data) / len(batch_x)
            else:
                rec, _ = model(data, False)
                loss_ae = criterion_ae(rec, data) / len(batch_x)

        if mode in ['both', 'clf']:
            rec_clean, logits = model(data, True)
            loss_clf = criterion_clf(logits, target)
            
            proba = torch.sigmoid(logits).detach().cpu().numpy()
            predictions = np.ones_like(proba, dtype=np.int32)
            predictions[proba < 0.5] = 0
            
            final_targets.append(target.detach().cpu().numpy())
            final_predictions.append(predictions)

        if mode == 'both':
            loss_total = loss_ae + lam_factor*loss_clf
            loss_clf_np = loss_clf.detach().cpu().numpy()
            loss_ae_np = loss_ae.detach().cpu().numpy()
            
            clf_train_loss.append(loss_clf_np)
            ae_train_loss.append(loss_ae_np)
            train_losses.append([loss_ae_np,loss_clf_np])
            
        elif mode == 'ae':
            loss_total = loss_ae
            loss_ae_np = loss_ae.detach().cpu().numpy()
            
            clf_train_loss.append(0.0)
            ae_train_loss.append(loss_ae_np)
            train_losses.append([loss_ae_np, 0.0])
            
        elif mode == 'clf':
            loss_total = loss_clf
            loss_clf_np = loss_clf.detach().cpu().numpy()
            
            clf_train_loss.append(loss_clf_np)
            ae_train_loss.append(0.0)
            train_losses.append([0.0,loss_clf_np])

        loss_total.backward()
        optimizer.step()
    
    if (final_targets is not None) and (final_predictions is not None):
        final_targets = np.concatenate(final_targets)
        final_predictions = np.concatenate(final_predictions)
        train_accuracy = np.mean(final_targets == final_predictions)

        return train_losses, np.mean(clf_train_loss), np.mean(ae_train_loss), train_accuracy
    else:
        return train_losses, np.mean(clf_train_loss), np.mean(ae_train_loss), None

def test(model, criterion, test_loader, 
         eval_classifier=False, num_batch=None):
    test_loss, n_test, correct = 0.0, 0, 0
    eval_loss = []
    all_predss=[]
    if eval_classifier:
        y_true, y_pred = [], []
    with torch.no_grad():
        model.eval()
#         bar = tqdm(test_loader, leave = False)
        for i,(batch_x,batch_y) in enumerate(test_loader):
            if num_batch is not None:
                if i >= num_batch:
                    continue
            data, target = batch_x.to(device), batch_y.to(device)
            rec, logits = model(data, eval_classifier)

#             test_loss += criterion(rec, data).detach().cpu().numpy() 
#             n_test += len(batch_x)
            test_loss = criterion(logits, target).detach().cpu().numpy() 
            eval_loss.append(test_loss)
            if eval_classifier:
                proba = torch.sigmoid(logits).detach().cpu().numpy()
                preds = np.ones_like(proba, dtype=np.int32)
                preds[proba < 0.5] = 0
                all_predss.extend(preds)###????
                y_arr = np.array(batch_y, dtype=np.int32)

                correct += np.sum(preds == y_arr)
                y_true.extend(y_arr.tolist())
                y_pred.extend(proba.tolist())
        mlp_acc,mlp_sens,mlp_spef = confusion(y_true,all_predss)
        metrics_dict = {'accuracy': np.round(mlp_acc, 4), 
                        'senstivity' : np.round(mlp_sens,4), 
                        'specificity' : np.round(mlp_spef,4), 
                        'loss' : np.round(np.mean(eval_loss),4)}
        
    return  metrics_dict

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
all_corr['Pitt_0050003'][0].shape

(2048,)

### Testing the data loader

In [ ]:
num_corr = len(all_corr[flist[0]][0])
print(num_corr)
# regions_inds = get_regs(flist,int(num_corr/4))
# sim_function = functools.partial(cal_similarity, lim=2)
regions_inds = None
train_loader=get_loader(data=all_corr, samples_list=flist, 
                                    batch_size=32, mode='train',
                                    augmentation=False, aug_factor=2, 
                                    num_neighbs=5, eig_data=None, similarity_fn=None, 
                                    verbose=True,regions=regions_inds)

2048


In [ ]:
i, l = next(iter(train_loader))
print(i.shape)
print(l.shape)

torch.Size([32, 2048])
torch.Size([32, 1])


In [ ]:
len(train_loader.dataset)

1035

### Main

In [ ]:
num_corr = len(all_corr[flist[0]][0])
print(num_corr)
# print(num_inpp)

2048


In [ ]:
if p_Method == "ASD-DiagNet" and p_mode == "whole":
    
    num_corr = len(all_corr[flist[0]][0])
    print("num_corr:  ",num_corr)
    
    start =time.time()
    batch_size = 8
    learning_rate_ae, learning_rate_clf = 0.0001, 0.0001
    num_epochs = 25

    p_bernoulli = None
    augmentation = p_augmentation
    use_dropout = False

    aug_factor = 2
    num_neighbs = 5
    lim4sim = 2
    n_lat = int(num_corr//4)
    print(n_lat)
    start= time.time()

    print('p_bernoulli: ', p_bernoulli)
    print('augmentaiton: ', augmentation, 'aug_factor: ', aug_factor, 
          'num_neighbs: ', num_neighbs, 'lim4sim: ', lim4sim)
    print('use_dropout: ', use_dropout, '\n')


    # sim_function = functools.partial(cal_similarity, lim=lim4sim)
    crossval_res_kol=[]
    y_arr = np.array([get_label(f) for f in flist])
    flist = np.array(flist)
    kk=0 

    # list to store metrics after each fold
    repeat_acc=[]
    repeat_sen=[]
    repeat_spec=[]
    repeat_loss=[]

    for rp in range(2):
        crossval_acc=[]
        crossval_sen=[]
        crossval_spec=[]
        crossval_loss=[]

        kf = StratifiedKFold(n_splits=p_fold, random_state=1, shuffle=True)
        np.random.shuffle(flist)
        y_arr = np.array([get_label(f) for f in flist])
        for kk,(train_index, test_index) in enumerate(kf.split(flist, y_arr)):
            #init wandb 
            NAME = f'asd-diagnet-sep-pca1024-cc200ho-fold-{kk+1}-rp-{rp+1}'
            ID = f'sep-pca1024-cc200ho-fold-{kk+1}-rp-{rp+1}'
            wandb.init(project = 'asd2', name=NAME, id=ID)

            train_samples, test_samples = flist[train_index], flist[test_index]
            verbose = (True if (kk == 0) else False)

            # regions_inds = get_regs(train_samples,int(num_corr/4))
            regions_inds = None

            # num_inpp = len(regions_inds)
            # n_lat = int(num_inpp/2)
            # n_lat = 4975
            n_lat = int(num_corr/2)
            train_loader=get_loader(data=all_corr, samples_list=train_samples, 
                                    batch_size=batch_size, mode='train',
                                    augmentation=augmentation, aug_factor=aug_factor, 
                                    num_neighbs=num_neighbs, eig_data=None, similarity_fn=None, 
                                    verbose=verbose,regions=regions_inds)

            test_loader=get_loader(data=all_corr, samples_list=test_samples, 
                                   batch_size=batch_size, mode='test', augmentation=False, 
                                   verbose=verbose,regions=regions_inds)

            # model = MTAutoEncoder(tied=True, num_inputs=num_inpp, num_latent=n_lat, use_dropout=use_dropout)
            model = MTAutoEncoder(tied=True, num_inputs=num_corr, num_latent=n_lat, use_dropout=use_dropout)
            model.to(device)
            criterion_ae = nn.MSELoss(reduction='sum')
            criterion_clf = nn.BCEWithLogitsLoss()
            optimizer = optim.SGD([{'params': model.fc_encoder.parameters(), 'lr': learning_rate_ae},
                                   {'params': model.classifier.parameters(), 'lr': learning_rate_clf}],
                                  momentum=0.9)


            for epoch in range(1, num_epochs+1):
                if epoch <= 20:
                    train_losses, clf_train_loss, ae_train_loss, train_acc = train(model, epoch, train_loader, p_bernoulli, mode='both')
                else:
                    train_losses, clf_train_loss, ae_train_loss, train_acc = train(model, epoch, train_loader, p_bernoulli, mode='clf')
                print(f'Epoch {epoch}/{num_epochs+1}')

                wandb.log({'Classifier Train loss': clf_train_loss, 'Train accuracy': train_acc, 'AE Train loss': ae_train_loss,})

                content = f' Classifier Train loss: {(clf_train_loss):.4f}, AE Train loss: {(ae_train_loss):.4f}, Train Acc: {(train_acc):.4f}'
                print(content)

            metrics_dict = test(model, criterion_clf, test_loader, eval_classifier=True)
            print("-----------------------------")
            print(f'Fold {kk+1}/{p_fold}')
            content = f'{metrics_dict}'
            print(content)
            print("-----------------------------")

            crossval_acc.append(metrics_dict['accuracy'])
            crossval_sen.append(metrics_dict['senstivity'])
            crossval_spec.append(metrics_dict['specificity'])
            crossval_loss.append(metrics_dict['loss'])
            
            #save the model after each fold
            
            recorder = {'optimizer': optimizer.state_dict(),
            'model': model.state_dict(),
            'fold' : kk+1,
            'repitition' : rp+1}

            torch.save(recorder, f'{NAME}.pt')
            
        print("*********************************")    
        print(f'Average Value after 10 Folds and repeats one {rp+1}------->')
        content = f'Accuracy: {np.round(np.mean(crossval_acc),4)}, Senstivity: {np.round(np.mean(crossval_sen),4)}, Specificity: {np.round(np.mean(crossval_spec),4)}, Loss: {np.round(np.mean(crossval_loss),4)}'
        print(content)
        print("*********************************") 
        
        repeat_acc.append(np.mean(crossval_acc))
        repeat_sen.append(np.mean(crossval_sen))
        repeat_spec.append(np.mean(crossval_spec))
        repeat_loss.append(np.mean(crossval_loss))
    
    print(f"Average Value after :",rp+1," Repeat:")
    content = f'Accuracy: {np.round(np.mean(repeat_acc),4)}, Senstivity: {np.round(np.mean(repeat_sen),4)}, Specificity: {np.round(np.mean(repeat_spec),4)}, Loss: {np.round(np.mean(repeat_loss),4)}'
    print(content)
        
    finish= time.time()
    print(finish-start)

num_corr:   2048
512
p_bernoulli:  None
augmentaiton:  False aug_factor:  2 num_neighbs:  5 lim4sim:  2
use_dropout:  False 



wandb: Currently logged in as: pindi-krishna (use `wandb login --relogin` to force relogin)


Epoch 1/26
 Classifier Train loss: 0.6790, AE Train loss: 654.0271, Train Acc: 0.5841
Epoch 2/26
 Classifier Train loss: 0.6472, AE Train loss: 496.2471, Train Acc: 0.6358
Epoch 3/26
 Classifier Train loss: 0.6206, AE Train loss: 409.0742, Train Acc: 0.6778
Epoch 4/26
 Classifier Train loss: 0.6002, AE Train loss: 346.2589, Train Acc: 0.6843
Epoch 5/26
 Classifier Train loss: 0.5830, AE Train loss: 298.3131, Train Acc: 0.7091
Epoch 6/26
 Classifier Train loss: 0.5654, AE Train loss: 259.5707, Train Acc: 0.7284
Epoch 7/26
 Classifier Train loss: 0.5510, AE Train loss: 228.2162, Train Acc: 0.7360
Epoch 8/26
 Classifier Train loss: 0.5384, AE Train loss: 202.2959, Train Acc: 0.7619
Epoch 9/26
 Classifier Train loss: 0.5238, AE Train loss: 180.4843, Train Acc: 0.7726
Epoch 10/26
 Classifier Train loss: 0.5130, AE Train loss: 161.6835, Train Acc: 0.7834
Epoch 11/26
 Classifier Train loss: 0.5013, AE Train loss: 145.7372, Train Acc: 0.7909
Epoch 12/26
 Classifier Train loss: 0.4904, AE Train

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Classifier Train loss,0.37542
Train accuracy,0.89763
AE Train loss,0.0
_step,24
_runtime,26
_timestamp,1602826507


...

Epoch 1/26
 Classifier Train loss: 0.6613, AE Train loss: 653.5106, Train Acc: 0.6261
Epoch 2/26
 Classifier Train loss: 0.6237, AE Train loss: 495.3823, Train Acc: 0.6789
Epoch 3/26
 Classifier Train loss: 0.5972, AE Train loss: 407.8715, Train Acc: 0.7091
Epoch 4/26
 Classifier Train loss: 0.5778, AE Train loss: 345.7660, Train Acc: 0.7306
Epoch 5/26
 Classifier Train loss: 0.5607, AE Train loss: 297.6375, Train Acc: 0.7435
Epoch 6/26
 Classifier Train loss: 0.5454, AE Train loss: 259.0612, Train Acc: 0.7554
Epoch 7/26
 Classifier Train loss: 0.5316, AE Train loss: 227.8134, Train Acc: 0.7672
Epoch 8/26
 Classifier Train loss: 0.5181, AE Train loss: 201.8455, Train Acc: 0.7866
Epoch 9/26
 Classifier Train loss: 0.5054, AE Train loss: 180.2889, Train Acc: 0.7909
Epoch 10/26
 Classifier Train loss: 0.4938, AE Train loss: 161.6709, Train Acc: 0.8039
Epoch 11/26
 Classifier Train loss: 0.4830, AE Train loss: 145.7711, Train Acc: 0.8114
Epoch 12/26
 Classifier Train loss: 0.4725, AE Train

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Classifier Train loss,0.35803
Train accuracy,0.89547
AE Train loss,0.0
_step,24
_runtime,17
_timestamp,1602826889


Classifier Train loss,█▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
Train accuracy,▁▂▃▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇███
AE Train loss,█▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
_step,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
_runtime,▁▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇████
_timestamp,▁▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇████


Epoch 1/26
 Classifier Train loss: 0.6913, AE Train loss: 652.3918, Train Acc: 0.5334
Epoch 2/26
 Classifier Train loss: 0.6568, AE Train loss: 495.6820, Train Acc: 0.6153
Epoch 3/26
 Classifier Train loss: 0.6295, AE Train loss: 409.1625, Train Acc: 0.6584
Epoch 4/26
 Classifier Train loss: 0.6061, AE Train loss: 345.9185, Train Acc: 0.6789
Epoch 5/26
 Classifier Train loss: 0.5888, AE Train loss: 298.0740, Train Acc: 0.6972
Epoch 6/26
 Classifier Train loss: 0.5713, AE Train loss: 259.5986, Train Acc: 0.7134
Epoch 7/26
 Classifier Train loss: 0.5551, AE Train loss: 228.3430, Train Acc: 0.7328
Epoch 8/26
 Classifier Train loss: 0.5392, AE Train loss: 202.3212, Train Acc: 0.7489
Epoch 9/26
 Classifier Train loss: 0.5274, AE Train loss: 180.4370, Train Acc: 0.7737
Epoch 10/26
 Classifier Train loss: 0.5135, AE Train loss: 161.8949, Train Acc: 0.7823
Epoch 11/26
 Classifier Train loss: 0.5006, AE Train loss: 145.8766, Train Acc: 0.7985
Epoch 12/26
 Classifier Train loss: 0.4896, AE Train

In [ ]:
model

MTAutoEncoder(
  (fc_encoder): Linear(in_features=76636, out_features=4975, bias=True)
  (classifier): Sequential(
    (0): Linear(in_features=4975, out_features=1, bias=True)
  )
)

### Temporary

In [ ]:
model = torch.load('./asd-diagnet-dim-cc200400-fold-7-rp-1.pt')
model

{'fold': 7,
 'model': OrderedDict([('fc_encoder.weight',
               tensor([[ 9.8775e-04,  3.5840e-03, -5.0433e-03,  ...,  2.2692e-03,
                         3.6612e-03, -2.2935e-03],
                       [ 1.7976e-03, -2.2401e-03,  2.3233e-03,  ..., -1.7451e-03,
                        -9.7384e-05,  1.1912e-03],
                       [ 1.0317e-03, -2.4080e-03, -3.8270e-03,  ...,  3.6958e-04,
                        -4.1012e-04,  7.2200e-04],
                       ...,
                       [-1.0043e-03, -1.5602e-03,  1.0857e-03,  ...,  6.5338e-04,
                        -5.5023e-03, -2.0529e-03],
                       [ 3.2595e-03, -3.7526e-04,  3.7544e-03,  ..., -1.8052e-03,
                        -1.0236e-03,  3.9574e-03],
                       [-4.2296e-03,  4.1404e-03,  2.6438e-03,  ..., -3.1233e-04,
                         4.6448e-03,  2.1198e-04]], device='cuda:0')),
              ('fc_encoder.bias',
               tensor([-0.0256,  0.0169, -0.0308,  ...,  0.0162

In [ ]:
del model

### Not required now 

In [ ]:

if p_Method == "ASD-DiagNet" and p_mode == "whole":
    
    num_corr = len(all_corr[flist[0]][0])
    print("num_corr:  ",num_corr)
    
    start =time.time()
    batch_size = 8
    learning_rate_ae, learning_rate_clf = 0.0001, 0.0001
    num_epochs = 25

    p_bernoulli = None
    augmentation = p_augmentation
    use_dropout = False

    aug_factor = 2
    num_neighbs = 5
    lim4sim = 2
    n_lat = int(num_corr/4)
    print(n_lat)
    start= time.time()

    print('p_bernoulli: ', p_bernoulli)
    print('augmentaiton: ', augmentation, 'aug_factor: ', aug_factor, 
          'num_neighbs: ', num_neighbs, 'lim4sim: ', lim4sim)
    print('use_dropout: ', use_dropout, '\n')


#     sim_function = functools.partial(cal_similarity, lim=lim4sim)
    
    
    y_arr = np.array([get_label(f) for f in flist])
    flist = np.array(flist)
    kk=0 
    
    # list to store metrics after each fold
    repeat_acc=[]
    repeat_sen=[]
    repeat_spec=[]
    repeat_loss=[]
    
    
    for rp in range(2):
        kf = StratifiedKFold(n_splits=p_fold, random_state=1, shuffle=True)
        np.random.shuffle(flist)
        y_arr = np.array([get_label(f) for f in flist])
    # list to store metrics after each fold
        crossval_acc=[]
        crossval_sen=[]
        crossval_spec=[]
        crossval_loss=[]
        
        for kk,(train_index, test_index) in enumerate(kf.split(flist, y_arr)):
            
            #init wandb 
            NAME = f'asd-diagnet-noaug-fold-{kk+1}-rp-{rp+1}'
            ID = f'noaug-fold-{kk+1}-rp-{rp+1}'
            wandb.init(project = 'asd2', name=NAME, id=ID)
            
            train_samples, test_samples = flist[train_index], flist[test_index]


            verbose = (True if (kk == 0) else False)

            regions_inds = get_regs(train_samples,int(num_corr/4))

            num_inpp = len(regions_inds)
            n_lat = int(num_inpp/2)
            
            # train_loader=get_loader(data=all_corr, samples_list=train_samples, 
            #                         batch_size=batch_size, mode='train',verbose=verbose,regions=regions_inds)

            # test_loader=get_loader(data=all_corr, samples_list=test_samples, 
            #                        batch_size = batch_size, mode='test', augmentation=False, 
            #                        verbose=verbose,regions=regions_inds)
            
            train_loader=get_loader(data=all_corr, samples_list=train_samples, 
                                    batch_size=batch_size, mode='train',
                                    augmentation=augmentation, aug_factor=aug_factor, 
                                    num_neighbs=num_neighbs, eig_data=None, similarity_fn=None, 
                                    verbose=verbose,regions=regions_inds)

            test_loader=get_loader(data=all_corr, samples_list=test_samples, 
                                   batch_size = batch_size, mode='test', augmentation=False, 
                                   verbose=verbose,regions=regions_inds)

            model = MTAutoEncoder(tied=True, num_inputs=num_inpp, num_latent=n_lat, use_dropout=use_dropout)
            model.to(device)
            wandb.watch(model, log='all')
            criterion_ae = nn.MSELoss(reduction='sum')
            criterion_clf = nn.CrossEntropyLoss()
            optimizer = optim.SGD([{'params': model.fc_encoder.parameters(), 'lr': learning_rate_ae},
                                   {'params': model.classifier.parameters(), 'lr': learning_rate_clf}],
                                  momentum=0.9)
            # optimizer = optim.Adam(model.parameters(),lr=learning_rate_clf)

            for epoch in range(1, num_epochs+1):
                if epoch <= 20:
                    train_losses, clf_train_loss, ae_train_loss, train_acc = train(model, epoch, train_loader, p_bernoulli, mode='both')
                else:
                    train_losses, clf_train_loss, ae_train_loss, train_acc = train(model, epoch, train_loader, p_bernoulli, mode='clf')
                print(f'Epoch {epoch}/{num_epochs+1}')

                wandb.log({'Classifier Train loss': clf_train_loss,
                            'Train accuracy': train_acc})

                content = f'Classifier Train loss: {(clf_train_loss):.4f}, Train Acc: {(train_acc):.4f}'
                print(content)

            metrics_dict = test(model, criterion_clf, test_loader, eval_classifier=True)
            print("-----------------------------")
            print(f'Fold {kk+1}/{p_fold}')
            content = f'{metrics_dict}'
            print(content)
            print("-----------------------------")
            
            crossval_acc.append(metrics_dict['accuracy'])
            crossval_sen.append(metrics_dict['senstivity'])
            crossval_spec.append(metrics_dict['specificity'])
            crossval_loss.append(metrics_dict['loss'])
            
            #save the model after each fold
            
            recorder = {'optimizer': optimizer.state_dict(),
            'model': model.state_dict(),
            'fold' : kk+1,
            'repitition' : rp+1}

            torch.save(recorder, f'{NAME}.pt')
            
        print("*********************************")    
        print(f'Average Value after 10 Folds and repeats one {rp+1}------->')
        content = f'Accuracy: {np.round(np.mean(crossval_acc),4)}, Senstivity: {np.round(np.mean(crossval_sen),4)}, Specificity: {np.round(np.mean(crossval_spec),4)}, Loss: {np.round(np.mean(crossval_loss),4)}'
        print(content)
        print("*********************************") 
        
        repeat_acc.append(np.mean(crossval_acc))
        repeat_sen.append(np.mean(crossval_sen))
        repeat_spec.append(np.mean(crossval_spec))
        repeat_loss.append(np.mean(crossval_loss))
    
    print(f"Average Value after :",rp+1," Repeat:")
    content = f'Accuracy: {np.round(np.mean(repeat_acc),4)}, Senstivity: {np.round(np.mean(repeat_sen),4)}, Specificity: {np.round(np.mean(repeat_spec),4)}, Loss: {np.round(np.mean(repeat_loss),4)}'
    print(content)
        
    finish= time.time()
    print(finish-start)




wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


num_corr:   19900
4975
p_bernoulli:  None
augmentaiton:  False aug_factor:  2 num_neighbs:  5 lim4sim:  2
use_dropout:  False 



RuntimeError: ignored

### OHSU results (2 repeats 5 fold) 

In [ ]:
if p_Method == "ASD-DiagNet" and p_mode == "percenter":
    
    num_corr = len(all_corr[flist[0]][0])

    flist = os.listdir(data_main_path)

    for f in range(len(flist)):
        flist[f] = get_key(flist[f])
    
    centers_dict = {}
    for f in flist:
        key = f.split('_')[0]

        if key not in centers_dict:
            centers_dict[key] = []
        centers_dict[key].append(f)

    

    flist = np.array(centers_dict[p_center])
    
    start =time.time()
    batch_size = 8
    learning_rate_ae, learning_rate_clf = 0.0001, 0.0001
    num_epochs = 25

    p_bernoulli = None
    augmentation = p_augmentation
    use_dropout = False

    aug_factor = 2
    num_neighbs = 5
    lim4sim = 2
    n_lat = int(num_corr/4)
    print(n_lat)
    start= time.time()

    print('p_bernoulli: ', p_bernoulli)
    print('augmentaiton: ', augmentation, 'aug_factor: ', aug_factor, 
          'num_neighbs: ', num_neighbs, 'lim4sim: ', lim4sim)
    print('use_dropout: ', use_dropout, '\n')


    sim_function = functools.partial(cal_similarity, lim=lim4sim)
    
    
    y_arr = np.array([get_label(f) for f in flist])
    flist = np.array(flist)
    kk=0
    
    # list to store metrics after each fold
    repeat_acc=[]
    repeat_sen=[]
    repeat_spec=[]
    repeat_loss=[]
    
    
    for rp in range(2):
        kf = StratifiedKFold(n_splits=p_fold, random_state=1, shuffle=True)
        np.random.shuffle(flist)
        y_arr = np.array([get_label(f) for f in flist])
    # list to store metrics after each fold
        crossval_acc=[]
        crossval_sen=[]
        crossval_spec=[]
        crossval_loss=[]
        for kk,(train_index, test_index) in enumerate(kf.split(flist, y_arr)):
            
            #init wandb 
            NAME = f'asd-diagnet-site--{p_center}-fold-{kk+1}-rp-{rp+1}'
            ID = f'fold-{kk+1}--rp-{rp+1}'
            wandb.init(project = 'asd1', name=NAME, id=ID)
            
            train_samples, test_samples = flist[train_index], flist[test_index]


            verbose = (True if (kk == 0) else False)

            regions_inds = get_regs(train_samples,int(num_corr/4))

            num_inpp = len(regions_inds)
            n_lat = int(num_inpp/2)
            
            train_loader=get_loader(data=all_corr, samples_list=train_samples, 
                                    batch_size=batch_size, mode='train',
                                    augmentation=augmentation, aug_factor=aug_factor, 
                                    num_neighbs=num_neighbs, eig_data=eig_data, similarity_fn=sim_function, 
                                    verbose=verbose,regions=regions_inds)

            test_loader=get_loader(data=all_corr, samples_list=test_samples, 
                                   batch_size=batch_size, mode='test', augmentation=False, 
                                   verbose=verbose,regions=regions_inds)
            model = MTAutoEncoder(tied=True, num_inputs=num_inpp, num_latent=n_lat, use_dropout=use_dropout)
            model.to(device)
            wandb.watch(model, log='all')
            criterion_ae = nn.MSELoss(reduction='sum')
            criterion_clf = nn.BCEWithLogitsLoss()
            optimizer = optim.SGD([{'params': model.fc_encoder.parameters(), 'lr': learning_rate_ae},
                                   {'params': model.classifier.parameters(), 'lr': learning_rate_clf}],
                                  momentum=0.9)
            for epoch in range(1, num_epochs+1):
                
                if epoch <= 20:
                    train_losses, clf_train_loss, ae_train_loss, train_acc = train(model, epoch, train_loader, 
                                                                        p_bernoulli, mode='both')
                else:
                    train_losses, clf_train_loss, ae_train_loss, train_acc = train(model, epoch, train_loader, 
                                                                        p_bernoulli, mode='clf')
                print(f'Epoch {epoch}/{num_epochs+1}')
                
                wandb.log({'Classifier Train loss': clf_train_loss,
                           'AE Train loss': ae_train_loss,
                            'Train accuracy': train_acc})
                
                content = f'Classifier Train loss: {(clf_train_loss):.4f}, AE Train loss: {(ae_train_loss):.4f}, Train Acc: {(train_acc):.4f}'
                print(content)

            metrics_dict = test(model, criterion_clf, test_loader, eval_classifier=True)
            print("-----------------------------")
            print(f'Fold {kk+1}/{p_fold}')
            content = f'{metrics_dict}'
            print(content)
            print("-----------------------------")
            
            crossval_acc.append(metrics_dict['accuracy'])
            crossval_sen.append(metrics_dict['senstivity'])
            crossval_spec.append(metrics_dict['specificity'])
            crossval_loss.append(metrics_dict['loss'])
            
            #save the model after each fold
            
            recorder = {'optimizer': optimizer.state_dict(),
            'model': model.state_dict(),
            'fold' : kk+1,
            'repitition' : rp+1}
            save_model = f'Saved-Models/{NAME}.pt'
            torch.save(recorder, save_model)
            
        print("*********************************")    
        print(f'Average Value after 5 Folds and repeats  {rp+1}------->')
        content = f'Accuracy: {np.round(np.mean(crossval_acc),4)}, Senstivity: {np.round(np.mean(crossval_sen),4)}, Specificity: {np.round(np.mean(crossval_spec),4)}, Loss: {np.round(np.mean(crossval_loss),4)}'
        print(content)
        print("*********************************") 
        
        repeat_acc.append(np.mean(crossval_acc))
        repeat_sen.append(np.mean(crossval_sen))
        repeat_spec.append(np.mean(crossval_spec))
        repeat_loss.append(np.mean(crossval_loss))
    
    print(f"Average Value after ",rp + 1," Repeat : ")
    content = f'Accuracy: {np.round(np.mean(repeat_acc),4)}, Senstivity: {np.round(np.mean(repeat_sen),4)}, Specificity: {np.round(np.mean(repeat_spec),4)}, Loss: {np.round(np.mean(repeat_loss),4)}'
    print(content)
        
    finish= time.time()
    print(finish-start)




4975
p_bernoulli:  None
augmentaiton:  True aug_factor:  2 num_neighbs:  5 lim4sim:  2
use_dropout:  False 



Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: pindi-krishna (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20200925_133043-fold-1--rp-1
wandb: Syncing run asd-diagnet-site--OHSU-fold-1-rp-1



Epoch 1/26
Classifier Train loss: 0.6893, AE Train loss: 644.2683, Train Acc: 0.4500
Epoch 2/26
Classifier Train loss: 0.6954, AE Train loss: 444.0914, Train Acc: 0.4500
Epoch 3/26
Classifier Train loss: 0.6681, AE Train loss: 382.0573, Train Acc: 0.6500
Epoch 4/26
Classifier Train loss: 0.6345, AE Train loss: 260.8163, Train Acc: 0.8250
Epoch 5/26
Classifier Train loss: 0.6034, AE Train loss: 167.0557, Train Acc: 0.8500
Epoch 6/26
Classifier Train loss: 0.5766, AE Train loss: 123.2638, Train Acc: 0.8500
Epoch 7/26
Classifier Train loss: 0.5196, AE Train loss: 70.7041, Train Acc: 0.9500
Epoch 8/26
Classifier Train loss: 0.4928, AE Train loss: 50.1150, Train Acc: 1.0000
Epoch 9/26
Classifier Train loss: 0.4733, AE Train loss: 30.4026, Train Acc: 1.0000
Epoch 10/26
Classifier Train loss: 0.4547, AE Train loss: 20.5647, Train Acc: 1.0000
Epoch 11/26
Classifier Train loss: 0.4280, AE Train loss: 11.9109, Train Acc: 1.0000
Epoch 12/26
Classifier Train loss: 0.4099, AE Train loss: 7.3965, T

wandb: Waiting for W&B process to finish, PID 3202
wandb: Program ended successfully.


wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20200925_133043-fold-1--rp-1/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20200925_133043-fold-1--rp-1/logs/debug-internal.log
wandb: Run summary:
wandb:   Classifier Train loss 0.2185116708278656
wandb:           AE Train loss 0.0
wandb:          Train accuracy 1.0
wandb:                   _step 24
wandb:                _runtime 9
wandb:              _timestamp 1601020857
wandb: Run history:
wandb:   Classifier Train loss ███▇▇▆▅▅▅▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁
wandb:           AE Train loss █▆▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          Train accuracy ▁▁▄▆▆▆▇██████████████████
wandb:                   _step ▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
wandb:                _runtime ▁▁▁▁▁▂▂▂▂▄▄▄▄▅▅▅▅▅▇▇▇▇▇██
wandb:              _timestamp ▁▁▁▁▁▂▂▂▂▄▄▄▄▅▅▅▅▅▇▇▇▇▇██
wandb: Synced 5 W&B file(s), 1 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: 
wandb: Syn

...

wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20200925_133227-fold-3--rp-2/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20200925_133227-fold-3--rp-2/logs/debug-internal.log
wandb: Run summary:
wandb:   Classifier Train loss 0.21351885795593262
wandb:           AE Train loss 0.0
wandb:          Train accuracy 1.0
wandb:                   _step 24
wandb:                _runtime 6
wandb:              _timestamp 1601020959
wandb: Run history:
wandb:   Classifier Train loss ███▇▇▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁
wandb:           AE Train loss █▆▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          Train accuracy ▁▁▃▅█████████████████████
wandb:                   _step ▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
wandb:                _runtime ▁▁▂▂▂▂▄▄▄▄▄▅▅▅▅▅▇▇▇▇█████
wandb:              _timestamp ▁▁▂▂▂▂▄▄▄▄▄▅▅▅▅▅▇▇▇▇█████
wandb: Synced 5 W&B file(s), 1 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: 
wandb: Sy


Epoch 1/26
Classifier Train loss: 0.6789, AE Train loss: 647.4289, Train Acc: 0.4500
Epoch 2/26
Classifier Train loss: 0.6708, AE Train loss: 434.0835, Train Acc: 0.6000
Epoch 3/26
Classifier Train loss: 0.6504, AE Train loss: 387.7741, Train Acc: 0.7750
Epoch 4/26
Classifier Train loss: 0.6206, AE Train loss: 261.6013, Train Acc: 0.9000
Epoch 5/26
Classifier Train loss: 0.5895, AE Train loss: 178.6221, Train Acc: 0.9000
Epoch 6/26
Classifier Train loss: 0.5464, AE Train loss: 130.0756, Train Acc: 0.9500
Epoch 7/26
Classifier Train loss: 0.5175, AE Train loss: 81.3757, Train Acc: 1.0000
Epoch 8/26
Classifier Train loss: 0.4858, AE Train loss: 55.9490, Train Acc: 1.0000
Epoch 9/26
Classifier Train loss: 0.4725, AE Train loss: 37.6683, Train Acc: 1.0000
Epoch 10/26
Classifier Train loss: 0.4406, AE Train loss: 24.6187, Train Acc: 1.0000
Epoch 11/26
Classifier Train loss: 0.4177, AE Train loss: 16.1353, Train Acc: 1.0000
Epoch 12/26
Classifier Train loss: 0.3977, AE Train loss: 9.8589, T

wandb: Waiting for W&B process to finish, PID 4265
wandb: Program ended successfully.


wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20200925_133240-fold-4--rp-2/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20200925_133240-fold-4--rp-2/logs/debug-internal.log
wandb: Run summary:
wandb:   Classifier Train loss 0.22409796714782715
wandb:           AE Train loss 0.0
wandb:          Train accuracy 1.0
wandb:                   _step 24
wandb:                _runtime 7
wandb:              _timestamp 1601020973
wandb: Run history:
wandb:   Classifier Train loss ███▇▇▆▆▅▅▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁
wandb:           AE Train loss █▆▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          Train accuracy ▁▃▅▇▇▇███████████████████
wandb:                   _step ▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
wandb:                _runtime ▁▂▂▂▂▃▃▃▃▃▅▅▅▅▆▆▆▆▆▇▇▇▇▇█
wandb:              _timestamp ▁▂▂▂▂▃▃▃▃▃▅▅▅▅▆▆▆▆▆▇▇▇▇▇█
wandb: Synced 5 W&B file(s), 1 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: 
wandb: Sy


Epoch 1/26
Classifier Train loss: 0.6968, AE Train loss: 639.1667, Train Acc: 0.6000
Epoch 2/26
Classifier Train loss: 0.6875, AE Train loss: 443.8496, Train Acc: 0.6500
Epoch 3/26
Classifier Train loss: 0.6510, AE Train loss: 391.1742, Train Acc: 0.6000
Epoch 4/26
Classifier Train loss: 0.6206, AE Train loss: 256.7818, Train Acc: 0.5500
Epoch 5/26
Classifier Train loss: 0.5771, AE Train loss: 179.9736, Train Acc: 0.6750
Epoch 6/26
Classifier Train loss: 0.5326, AE Train loss: 119.9725, Train Acc: 0.8500
Epoch 7/26
Classifier Train loss: 0.5207, AE Train loss: 82.2343, Train Acc: 0.9000
Epoch 8/26
Classifier Train loss: 0.4895, AE Train loss: 53.2690, Train Acc: 0.9500
Epoch 9/26
Classifier Train loss: 0.4638, AE Train loss: 35.1413, Train Acc: 0.9750
Epoch 10/26
Classifier Train loss: 0.4413, AE Train loss: 23.1502, Train Acc: 0.9750
Epoch 11/26
Classifier Train loss: 0.4214, AE Train loss: 16.7047, Train Acc: 1.0000
Epoch 12/26
Classifier Train loss: 0.3873, AE Train loss: 10.5827, 

In [ ]:

# if p_Method == "ASD-DiagNet" and p_mode == "percenter":
#     num_corr = len(all_corr[flist[0]][0])

#     flist = os.listdir(data_main_path)

#     for f in range(len(flist)):
#         flist[f] = get_key(flist[f])
    
#     centers_dict = {}
#     for f in flist:
#         key = f.split('_')[0]

#         if key not in centers_dict:
#             centers_dict[key] = []
#         centers_dict[key].append(f)

    

#     flist = np.array(centers_dict[p_center])
    
#     start =time.time()
#     #flist = np.array(sorted(os.listdir(data_main_path)))
#     batch_size = 8
#     learning_rate_ae, learning_rate_clf = 0.0001, 0.0001
#     num_epochs = 25

#     p_bernoulli = None
#     augmentation = p_augmentation
#     use_dropout = False

#     aug_factor = 2
#     num_neighbs = 5
#     lim4sim = 2
#     n_lat = int(num_corr/4)


#     print('p_bernoulli: ', p_bernoulli)
#     print('augmentaiton: ', augmentation, 'aug_factor: ', aug_factor, 
#           'num_neighbs: ', num_neighbs, 'lim4sim: ', lim4sim)
#     print('use_dropout: ', use_dropout, '\n')


#     sim_function = functools.partial(cal_similarity, lim=lim4sim)
#     all_rp_res=[]
#     y_arr = np.array([get_label(f) for f in flist])

#     kk=0 
#     crossval_res_kol_kol=[]
#     for rp in range(10):
#         print("========================")
#         crossval_res_kol = []
#         start= time.time()
#         kf = StratifiedKFold(n_splits=p_fold)
#         #np.random.shuffle(flist)
#         y_arr = np.array([get_label(f) for f in flist])
#         for kk,(train_index, test_index) in enumerate(kf.split(flist, y_arr)):
        
#             train_samples, test_samples = flist[train_index], flist[test_index]

#             verbose = (True if (kk == 0) else False)

#             regions_inds = get_regs(train_samples,int(num_corr/4))
#             num_inpp = len(regions_inds)
#             n_lat = int(num_inpp/2)
#             num_inpp = len(regions_inds)
#             train_loader=get_loader(data=all_corr, samples_list=train_samples, 
#                                     batch_size=batch_size, mode='train',
#                                     augmentation=augmentation, aug_factor=aug_factor, 
#                                     num_neighbs=num_neighbs, eig_data=eig_data, similarity_fn=sim_function, 
#                                     verbose=verbose,regions=regions_inds)

#             test_loader=get_loader(data=all_corr, samples_list=test_samples, 
#                                    batch_size=batch_size, mode='test', augmentation=False, 
#                                    verbose=verbose,regions=regions_inds)

#             model = MTAutoEncoder(tied=True, num_inputs=num_inpp, num_latent=n_lat, use_dropout=use_dropout)
#             model.to(device)
#             criterion_ae = nn.MSELoss(reduction='sum')
#             criterion_clf = nn.BCEWithLogitsLoss()
#             optimizer = optim.SGD([{'params': model.fc_encoder.parameters(), 'lr': learning_rate_ae},
#                                    {'params': model.classifier.parameters(), 'lr': learning_rate_clf}],
#                                   momentum=0.9)

#             for epoch in range(1, num_epochs+1):
#                 if epoch <= 20:
#                     train_losses = train(model, epoch, train_loader, p_bernoulli, mode='both')
#                 else:
#                     train_losses = train(model, epoch, train_loader, p_bernoulli, mode='clf')


#             res_mlp = test(model, criterion_ae, test_loader, eval_classifier=True)
#             #print("fold",kk+1,":",test(model, criterion_ae, test_loader, eval_classifier=True))
#             crossval_res_kol.append(res_mlp)
#         print("Result of repeat ",rp,":")
#         print(np.mean(np.array(crossval_res_kol),axis = 0))
#         all_rp_res.append(np.mean(np.array(crossval_res_kol),axis = 0))
#         finish= time.time()

#         print("Running time:",finish-start)
#     print("Avergae result of 10 repeats: ",np.mean(np.array(all_rp_res),axis = 0))

In [ ]:
# if p_Method != "ASD-DiagNet" and p_mode == "whole":
    
#     clf = SVC(gamma = 'auto') if  p_Method == 'SVM' else RandomForestClassifier(n_estimators=100)
#     overall_result = []
#     for rp in range(10):
#         kf = StratifiedKFold(n_splits=p_fold, random_state=1, shuffle=True)
#         np.random.shuffle(flist)
#         y_arr = np.array([get_label(f) for f in flist])
#         res = []
#         for kk,(train_index, test_index) in enumerate(kf.split(flist, y_arr)):
#             train_samples, test_samples = np.array(flist)[train_index], np.array(flist)[test_index]
#             train_data = []
#             train_labels = []
#             test_data = []
#             test_labels = []

#             for i in train_samples:
#                 train_data.append(all_corr[i][0])
#                 train_labels.append(all_corr[i][1])

#             for i in test_samples:
#                 test_data.append(all_corr[i][0])
#                 test_labels.append(all_corr[i][1])

            
#             clf.fit(train_data,train_labels)
#             pr = clf.predict(test_data)
#             res.append(confusion(test_labels,pr))
        
#         print("repeat: ",rp,np.mean(res, axis=0).tolist())
#         overall_result.append(np.mean(res, axis=0).tolist())   
#     print("---------------Result of repeating 10 times-------------------")
#     print(np.mean(np.array(overall_result), axis=0).tolist())

In [ ]:
# random.seed(19)
# np.random.seed(19)
# if p_Method != "ASD-DiagNet" and p_mode == "percenter":
    
#     clf = SVC(gamma = 'auto') if  p_Method == 'SVM' else RandomForestClassifier(n_estimators=100)
#     overall_result = []
#     for rp in range(10):
#         kf = StratifiedKFold(n_splits=p_fold, random_state=1, shuffle=True)
#         np.random.shuffle(flist)
#         y_arr = np.array([get_label(f) for f in flist])
#         res = []
#         for kk,(train_index, test_index) in enumerate(kf.split(flist, y_arr)):
#             train_samples, test_samples = np.array(flist)[train_index], np.array(flist)[test_index]
#             train_data = []
#             train_labels = []
#             test_data = []
#             test_labels = []

#             for i in train_samples:
#                 train_data.append(all_corr[i][0])
#                 train_labels.append(all_corr[i][1])

#             for i in test_samples:
#                 test_data.append(all_corr[i][0])
#                 test_labels.append(all_corr[i][1])

#             clf.fit(train_data,train_labels)
#             pr = clf.predict(test_data)
#             res.append(confusion(test_labels,pr))
        
#         print("repeat: ",rp,np.mean(res, axis=0).tolist())
#         overall_result.append(np.mean(res, axis=0).tolist())   
#     print("---------------Result of repeating 10 times for: ",p_center,"-------------------")
#     print(np.mean(np.array(overall_result), axis=0).tolist())